In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pytest
import os

import FINE as fn
import numpy as np
import pandas as pd

import sys
#sys.path.append(os.path.join(os.path.dirname(__file__),'..','examples','Multi-regional_Energy_System_Workflow'))
#from getData import getData
import xlrd

import geopandas as gpd

import pandas as pd
import os
import xlrd

## Data - minimal_test_esM

In [3]:
numberOfTimeSteps = 4
hoursPerTimeStep = 2190

# Create an energy system model instance 
esM = fn.EnergySystemModel(locations={'ElectrolyzerLocation', 'IndustryLocation'}, 
                            commodities={'electricity', 'hydrogen'}, 
                            numberOfTimeSteps=numberOfTimeSteps,
                            commodityUnitsDict={'electricity': r'kW$_{el}$', 'hydrogen': r'kW$_{H_{2},LHV}$'},
                            hoursPerTimeStep=hoursPerTimeStep, costUnit='1 Euro', 
                            lengthUnit='km', 
                            verboseLogLevel=2)

# time step length [h]
timeStepLength = numberOfTimeSteps * hoursPerTimeStep

### Buy electricity at the electricity market
costs = pd.DataFrame([np.array([ 0.05, 0., 0.1, 0.051,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
revenues = pd.DataFrame([np.array([ 0., 0.01, 0., 0.,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
maxpurchase = pd.DataFrame([np.array([1e6, 1e6, 1e6, 1e6,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Source(esM=esM, name='Electricity market', commodity='electricity', 
                    hasCapacityVariable=False, operationRateMax = maxpurchase,
                    commodityCostTimeSeries = costs,  
                    commodityRevenueTimeSeries = revenues,  
                    )) # eur/kWh

### Electrolyzers
esM.add(fn.Conversion(esM=esM, name='Electrolyzers', physicalUnit=r'kW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=500, # euro/kW
                      opexPerCapacity=500*0.025, 
                      interestRate=0.08,
                      economicLifetime=10))

### Hydrogen filled somewhere
esM.add(fn.Storage(esM=esM, name='Pressure tank', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   stateOfChargeMin=0.33, 
                   investPerCapacity=0.5, # eur/kWh
                   interestRate=0.08,
                   economicLifetime=30))

### Hydrogen pipelines
esM.add(fn.Transmission(esM=esM, name='Pipelines', commodity='hydrogen',
                        hasCapacityVariable=True,
                        investPerCapacity=0.177, 
                        interestRate=0.08, 
                        economicLifetime=40))

### Industry site
demand = pd.DataFrame([np.array([0., 0., 0., 0.,]), np.array([6e3, 6e3, 6e3, 6e3,]),],
                index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Sink(esM=esM, name='Industry site', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix = demand,
                ))

In [4]:
minimal_test_esM = esM

In [5]:
esM.getComponentAttribute('Pipelines', 'investPerCapacity')

IndustryLocation_ElectrolyzerLocation    0.0885
ElectrolyzerLocation_IndustryLocation    0.0885
dtype: float64

In [7]:
type(esM.getComponentAttribute('Industry site', 'operationRateFix'))

pandas.core.frame.DataFrame

## test 

In [8]:
#nc_path = os.path.join(os.path.dirname(__file__), '../data/ds_multinode.nc4')

esm_dict, component_dict = fn.dictIO.exportToDict(esM)

In [9]:
esm_dict

{'locations': {'ElectrolyzerLocation', 'IndustryLocation'},
 'commodities': {'electricity', 'hydrogen'},
 'commodityUnitsDict': {'electricity': 'kW$_{el}$',
  'hydrogen': 'kW$_{H_{2},LHV}$'},
 'numberOfTimeSteps': 4,
 'hoursPerTimeStep': 2190,
 'costUnit': '1 Euro',
 'lengthUnit': 'km',
 'verboseLogLevel': 2}

In [10]:
component_dict

{'Source': {'Electricity market': {'name': 'Electricity market',
   'commodity': 'electricity',
   'hasCapacityVariable': False,
   'capacityVariableDomain': 'continuous',
   'capacityPerPlantUnit': 1,
   'hasIsBuiltBinaryVariable': False,
   'bigM': None,
   'operationRateMax': None,
   'operationRateFix': None,
   'tsaWeight': 1,
   'commodityLimitID': None,
   'yearlyLimit': None,
   'locationalEligibility': ElectrolyzerLocation    1.0
   IndustryLocation        0.0
   dtype: float64,
   'capacityMin': None,
   'capacityMax': None,
   'partLoadMin': None,
   'sharedPotentialID': None,
   'linkedQuantityID': None,
   'capacityFix': None,
   'isBuiltFix': None,
   'investPerCapacity': ElectrolyzerLocation    0.0
   IndustryLocation        0.0
   dtype: float64,
   'investIfBuilt': ElectrolyzerLocation    0.0
   IndustryLocation        0.0
   dtype: float64,
   'opexPerOperation': ElectrolyzerLocation    0.0
   IndustryLocation        0.0
   dtype: float64,
   'commodityCost': Electrol

In [9]:
component_dict

{'Source': {'Electricity market': {'name': 'Electricity market',
   'commodity': 'electricity',
   'hasCapacityVariable': False,
   'capacityVariableDomain': 'continuous',
   'capacityPerPlantUnit': 1,
   'hasIsBuiltBinaryVariable': False,
   'bigM': None,
   'operationRateMax':                  ElectrolyzerLocation  IndustryLocation
   Period TimeStep                                        
   0      0                 2.190000e+09               0.0
          1                 2.190000e+09               0.0
          2                 2.190000e+09               0.0
          3                 2.190000e+09               0.0,
   'operationRateFix': None,
   'tsaWeight': 1,
   'commodityLimitID': None,
   'yearlyLimit': None,
   'locationalEligibility': ElectrolyzerLocation    1.0
   IndustryLocation        0.0
   dtype: float64,
   'capacityMin': None,
   'capacityMax': None,
   'partLoadMin': None,
   'sharedPotentialID': None,
   'capacityFix': None,
   'isBuiltFix': None,
   'investPe

In [10]:
ds_extracted = fn.xarray_io.dimensional_data_to_xarray_dataset(esm_dict, component_dict)



In [11]:
ds_extracted

<xarray.Dataset>
Dimensions:                       (Period: 1, TimeStep: 4, component: 5, space: 2, space_2: 2)
Coordinates:
  * component                     (component) object 'Conversion, Electrolyze...
  * Period                        (Period) int64 0
  * TimeStep                      (TimeStep) int64 0 1 2 3
  * space                         (space) object 'ElectrolyzerLocation' 'Indu...
  * space_2                       (space_2) object 'ElectrolyzerLocation' 'In...
Data variables:
    operationRateMax              (component, Period, TimeStep, space) float64 ...
    commodityCostTimeSeries       (component, Period, TimeStep, space) float64 ...
    commodityRevenueTimeSeries    (component, Period, TimeStep, space) float64 ...
    operationRateFix              (component, Period, TimeStep, space) float64 ...
    2d_locationalEligibility      (component, space, space_2) float64 nan ......
    2d_investPerCapacity          (component, space, space_2) float64 nan ......
    2d_investIfBuilt              (component, space, space_2) float64 nan ......
    2d_opexPerOperation           (component, space, space_2) float64 nan ......
    2d_opexPerCapacity            (component, space, space_2) float64 nan ......
    2d_opexIfBuilt                (component, space, space_2) float64 nan ......
    2d_QPcostScale                (component, space, space_2) float64 nan ......
    2d_interestRate               (component, space, space_2) float64 nan ......
    2d_economicLifetime           (component, space, space_2) float64 nan ......
    2d_technicalLifetime          (component, space, space_2) float64 nan ......
    2d_losses                     (component, space, space_2) float64 nan ......
    2d_distances                  (component, space, space_2) float64 nan ......
    1d_locationalEligibility      (component, space) float64 1.0 1.0 ... nan nan
    1d_investPerCapacity          (component, space) float64 500.0 500.0 ... nan
    1d_investIfBuilt              (component, space) float64 0.0 0.0 ... nan nan
    1d_opexPerOperation           (component, space) float64 0.0 0.0 ... nan nan
    1d_commodityCost              (component, space) float64 nan nan ... nan nan
    1d_commodityRevenue           (component, space) float64 nan nan ... nan nan
    1d_opexPerCapacity            (component, space) float64 12.5 12.5 ... nan
    1d_opexIfBuilt                (component, space) float64 0.0 0.0 ... nan nan
    1d_QPcostScale                (component, space) float64 0.0 0.0 ... nan nan
    1d_interestRate               (component, space) float64 0.08 0.08 ... nan
    1d_economicLifetime           (component, space) float64 10.0 10.0 ... nan
    1d_technicalLifetime          (component, space) float64 10.0 10.0 ... nan
    1d_opexPerChargeOperation     (component, space) float64 nan nan ... nan nan
    1d_opexPerDischargeOperation  (component, space) float64 nan nan ... nan nan

In [12]:
expected_number_of_locations = len(minimal_test_esM.locations)
expected_number_of_locations

2

In [13]:
actual_number_of_locations = len(ds_extracted.space)
actual_number_of_locations

2